# COVID19 - Countries

In [1]:
import os
import ray

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
from tabulate import tabulate
from IPython.display import HTML

In [3]:
# Download data
import get_data as gd
LoadData=True

if LoadData:
    gd.get_data()

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The sum of all people accounted is equal to population considered:

$$s + e + a + i + r + k = N$$

The derivate is:

$$\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

N: the population considered

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific Country

In [4]:
from IPython.display import display, HTML, Markdown

changeCSV=False
adjustParam=True
paramSave=False
selectCountries=False

## Different Options to Load Param.CSV file

In [5]:
paramOpt=1
model = "YaboxIC"    

#original parameters file
if paramOpt==0:
    paramFile="data/param.csv"

#optimized parameters file based on history    
if paramOpt==1:
    paramFile="data/param_optimized_Yabox_HistMin.csv"

#optimized parameters file based on final results        
if paramOpt==2:
    paramFile="param_optimized_Yabox.csv"
    

In [6]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.a0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH
0,Brazil,03/01/20,200,5351475,0,0,94,232,195,246,0.2626,0.5071,0.5957
1,China,01/27/20,200,327516,0,0,200,100,50,50,0.1903,0.6452,0.6084
2,Italy,02/28/20,200,43486447,0,0,106,164,64,3,0.3301,0.0646,0.3313
3,US,02/18/20,200,89254715,0,0,248,149,221,114,0.1641,0.7578,0.2689
4,India,03/10/20,200,42221513,0,0,84,142,156,228,0.1575,0.2788,0.6303


## Modify param.csv file to fit better the data

In [7]:
if changeCSV and (paramOpt==1):
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    dfparam['s0'] = dfparam['s0'].astype(float)
    dfparam['i0'] = dfparam['i0'].astype(float)
    dfparam['d0'] = dfparam['d0'].astype(float)
    dfparam.loc[dfparam.country=='Italy','s0'] = dfparam.loc[dfparam.country=='Italy','s0']*1.8
    dfparam.loc[dfparam.country=='China','s0'] = dfparam.loc[dfparam.country=='China','s0']*1.5
    dfparam.loc[dfparam.country=='India','s0'] = dfparam.loc[dfparam.country=='India','s0']*12/100
    dfparam.loc[dfparam.country=='US','s0'] = dfparam.loc[dfparam.country=='US','s0']*8/10
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    

# if adjustParam:
#     sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
#     dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectCountries and paramOpt==0:
    countries=['Brazil', 
         'US']

if not selectCountries:
    countries=dfparam.country    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)

if paramSave:
    dfparam.to_csv("new_"+paramFile)    

display(Markdown("## Modified Initial Parameters"))

display(dfparam)

## Modified Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH
0,Brazil,03/01/20,200,5351475,0,0,94,232,195,246,0.2626,0.5071,0.5957
1,China,01/27/20,200,327516,0,0,200,100,50,50,0.1903,0.6452,0.6084
2,Italy,02/28/20,200,43486447,0,0,106,164,64,3,0.3301,0.0646,0.3313
3,US,02/18/20,200,89254715,0,0,248,149,221,114,0.1641,0.7578,0.2689
4,India,03/10/20,200,42221513,0,0,84,142,156,228,0.1575,0.2788,0.6303


# Solver Loading and Version

In [8]:
%reload_ext autoreload
%autoreload 2
underNotif=True
import LearnerYabox_v5 as Learner


Yabox version:  1.1.0


2020-10-24 16:22:55,236	INFO resource_spec.py:212 -- Starting Ray with 229.98 GiB memory available for workers and up to 70.88 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-24 16:22:55,565	INFO services.py:1170 -- View the Ray dashboard at localhost:8268


In [9]:
allCountries=True
cleanRecovered=False
version="2"

if allCountries:
    display(countries)
else:
    countrySelected="Italy"
    display(countrySelected)

0    Brazil
1     China
2     Italy
3        US
4     India
Name: country, dtype: object

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
results=[]
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        current = datetime.strptime(parameters[0], '%m/%d/%y')
        parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
        if country=='Italy':
            endDateStr = "10/15/20"
        else:
            endDate = date.today() + timedelta(days = -2)
            endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
        endDate = datetime.strptime(endDateStr, '%m/%d/%y')
        end_date= datetime.strftime(endDate, '%-m/%-d/%y')
        f=Learner.Learner.remote(country, *parameters, end_date, 
                                 cleanRecovered, version,underNotif=underNotif)
        result = f.train.remote() 
        results.append(result)
else:
    country = countrySelected 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    current = datetime.strptime(parameters[0], '%m/%d/%y')
    parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
    if country=='Italy':
        endDateStr = "9/1/20"
    else:
        endDate = date.today() + timedelta(days = -2)
        endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
    endDate = datetime.strptime(endDateStr, '%m/%d/%y')
    end_date= datetime.strftime(endDate, '%-m/%-d/%y')  
    f=Learner.Learner.remote(country, *parameters, end_date, 
                             cleanRecovered, version,underNotif=underNotif)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

2020-10-24 16:22:56,431	WARNING worker.py:1090 -- WARNING: 3 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-24 16:22:56,830	WARNING worker.py:1090 -- WARNING: 4 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-24 16:22:57,245	WARNING worker.py:1090 -- WARNING: 5 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

 27%|██▋       | 5612925/21125000.0 [43:42<48:14, 5359.95it/s]
 27%|██▋       | 5690251/21125000.0 [43:42<44:39, 5759.73it/s]
 33%|███▎      | 6976980/21125000.0 [43:43<33:40, 7002.93it/s]
 29%|██▉       | 6116253/21125000.0 [43:44<47:22, 5279.63it/s]
 27%|██▋       | 5616276/21125000.0 [43:42<48:02, 5380.36it/s]
 27%|██▋       | 5693625/21125000.0 [43:43<44:13, 5815.73it/s]
 21%|██        | 4462578/21125000.0 [43:44<1:18:46, 3525.42it/s]
 33%|███▎      | 6980716/21125000.0 [43:43<34:35, 6816.24it/s]
 27%|██▋       | 5619628/21125000.0 [43:43<48:00, 5382.55it/s]
 29%|██▉       | 6119751/21125000.0 [43:44<48:04, 5202.28it/s]
 27%|██▋       | 5697000/21125000.0 [43:43<44:17, 5806.03it/s]
 33%|███▎      | 6984453/21125000.0 [43:44<35:33, 6626.91it/s]
 21%|██        | 4465566/21125000.0 [43:45<1:20:23, 3454.09it/s]
 27%|██▋       | 5700376/21125000.0 [43:44<44:14, 5811.55it/s]
 27%|██▋       | 5622981/21125000.0 [43:43<48:20, 5344.27it/s]
 29%|██▉       | 6123250/21125000.0 [43:45<48:15, 5

 21%|██▏       | 4522528/21125000.0 [44:01<1:20:03, 3456.01it/s]
 27%|██▋       | 5710510/21125000.0 [44:00<47:27, 5413.92it/s]
 27%|██▋       | 5791906/21125000.0 [44:00<47:01, 5433.88it/s]
 29%|██▉       | 6211050/21125000.0 [44:02<45:51, 5419.85it/s]
 27%|██▋       | 5713890/21125000.0 [44:00<46:38, 5507.90it/s]
 21%|██▏       | 4525536/21125000.0 [44:02<1:20:20, 3443.84it/s]
 27%|██▋       | 5795310/21125000.0 [44:01<45:28, 5617.76it/s]
 29%|██▉       | 6214575/21125000.0 [44:02<45:56, 5409.85it/s]
 27%|██▋       | 5717271/21125000.0 [44:01<46:53, 5476.73it/s]
 27%|██▋       | 5798715/21125000.0 [44:01<45:18, 5638.79it/s]
 21%|██▏       | 4528545/21125000.0 [44:02<1:19:51, 3463.65it/s]
 29%|██▉       | 6218101/21125000.0 [44:03<47:35, 5220.43it/s]
 27%|██▋       | 5720653/21125000.0 [44:01<46:46, 5488.78it/s]
 27%|██▋       | 5802121/21125000.0 [44:02<45:17, 5638.35it/s]
 29%|██▉       | 6221628/21125000.0 [44:04<46:51, 5300.68it/s]
 27%|██▋       | 5724036/21125000.0 [44:02<47:28,

 22%|██▏       | 4598028/21125000.0 [44:22<1:17:08, 3570.88it/s]
 34%|███▍      | 7165005/21125000.0 [44:22<32:53, 7074.22it/s]
 28%|██▊       | 5829405/21125000.0 [44:21<44:42, 5701.60it/s]
 30%|██▉       | 6320790/21125000.0 [44:23<45:34, 5414.58it/s]
 34%|███▍      | 7168791/21125000.0 [44:22<32:45, 7101.00it/s]
 28%|██▊       | 5832820/21125000.0 [44:21<45:12, 5638.35it/s]
 30%|██▉       | 6324346/21125000.0 [44:23<43:30, 5669.96it/s]
 22%|██▏       | 4601061/21125000.0 [44:23<1:17:20, 3561.04it/s]
 34%|███▍      | 7172578/21125000.0 [44:23<32:13, 7214.98it/s]
 28%|██▊       | 5836236/21125000.0 [44:22<45:09, 5642.35it/s]
 28%|██▊       | 5839653/21125000.0 [44:22<5:53:35, 720.47it/s]
 30%|██▉       | 6327903/21125000.0 [44:24<45:15, 5449.89it/s]
 22%|██▏       | 4604095/21125000.0 [44:24<1:16:36, 3594.22it/s]
 34%|███▍      | 7176366/21125000.0 [44:23<31:53, 7290.53it/s]
 28%|██▊       | 5843071/21125000.0 [44:23<4:19:14, 982.48it/s]
 28%|██▊       | 5839653/21125000.0 [44:23<45:3

 30%|███       | 6409990/21125000.0 [44:39<44:25, 5521.40it/s]
 22%|██▏       | 4658878/21125000.0 [44:39<1:16:32, 3585.70it/s]
 34%|███▍      | 7282836/21125000.0 [44:39<35:43, 6458.29it/s]
 28%|██▊       | 5932290/21125000.0 [44:38<44:58, 5630.93it/s]
 28%|██▊       | 5925403/21125000.0 [44:38<47:02, 5385.33it/s]
 30%|███       | 6413571/21125000.0 [44:40<43:37, 5621.38it/s]
 22%|██▏       | 4661931/21125000.0 [44:40<1:16:41, 3578.13it/s]
 28%|██▊       | 5935735/21125000.0 [44:39<44:04, 5743.09it/s]
 34%|███▍      | 7286653/21125000.0 [44:39<35:20, 6526.39it/s]
 28%|██▊       | 5928846/21125000.0 [44:39<47:40, 5313.31it/s]
 30%|███       | 6417153/21125000.0 [44:40<45:12, 5422.21it/s]
 35%|███▍      | 7290471/21125000.0 [44:40<35:15, 6540.48it/s]
 28%|██▊       | 5939181/21125000.0 [44:40<44:35, 5676.60it/s]
 22%|██▏       | 4664985/21125000.0 [44:41<1:16:38, 3579.32it/s]
 28%|██▊       | 5932290/21125000.0 [44:39<48:34, 5213.55it/s]
 30%|███       | 6420736/21125000.0 [44:41<45:40,

 31%|███       | 6503421/21125000.0 [44:56<44:29, 5476.72it/s]
 28%|██▊       | 6015246/21125000.0 [44:54<44:43, 5631.51it/s]
 35%|███▌      | 7397781/21125000.0 [44:55<31:20, 7298.56it/s]
 22%|██▏       | 4720128/21125000.0 [44:56<1:15:00, 3645.26it/s]
 29%|██▊       | 6029128/21125000.0 [44:55<43:22, 5801.34it/s]
 31%|███       | 6507028/21125000.0 [44:57<42:44, 5700.44it/s]
 28%|██▊       | 6018715/21125000.0 [44:55<45:14, 5564.56it/s]
 35%|███▌      | 7401628/21125000.0 [44:56<33:30, 6827.26it/s]
 29%|██▊       | 6032601/21125000.0 [44:56<42:41, 5891.36it/s]
 22%|██▏       | 4723201/21125000.0 [44:57<1:14:57, 3646.97it/s]
 31%|███       | 6510636/21125000.0 [44:57<43:40, 5576.08it/s]
 35%|███▌      | 7405476/21125000.0 [44:57<32:47, 6973.33it/s]
 29%|██▊       | 6022185/21125000.0 [44:56<44:52, 5608.42it/s]
 29%|██▊       | 6036075/21125000.0 [44:56<43:08, 5828.84it/s]
 35%|███▌      | 7409325/21125000.0 [44:57<33:12, 6884.76it/s]
 22%|██▏       | 4726275/21125000.0 [44:58<1:14:26,

In [ ]:
ray.shutdown()

# Optimized Parameters and Curves Adjust

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'sub', 'subRec', 'subDth']
dfVars = pd.DataFrame(data=results, columns=cols)
if allCountries:
    dfVars['country']=dfparam.country
else:
    dfVars['country']=countrySelected
    display(countrySelected)

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'country','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]

display(Markdown("## Under Notification and Adjusts"))
display(dfVars.iloc[:,12:16])

In [ ]:
cols=['country','beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]
dfVars.to_csv('./results/underNotif_vars_'+version2+'.csv', sep=",")

display(Markdown("## Simulation Parameters"))

cols=['index','country','$\\beta_0$', '$\\beta_{01}$', 'startT', '$\\beta_2$', '$\\sigma$', '$\\sigma_2$', 
          '$\\sigma_3$', 'a', 'b', 'c', 'd', '$\\mu$','sub', 'subRec', 'subDth']
    
HTML(tabulate(dfVars.iloc[:,:13], headers= cols, floatfmt='.3e', tablefmt='html'))

# Plots

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots_v2 as cp

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

## Plot Parameters

In [ ]:
#main country for analysis
countryMain = "Brazil"

#select countries for plotting
countries4Plot=['China',
               'US',
               'India',
                countryMain,
               'Italy'
               ]

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

#all Countries for plotting
countries=dfparam.country

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

## Plot Loop

In [ ]:
#plots one country or all countries  
if opt==5:
    allCountries=True
else:
    allCountries=False
    
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        #calcula data máxima dos gráficos
        #200 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate='9/22/20'
        maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
        maxDateStr = maxDate.strftime("%-m/%-d/%y")
        cp.covid_plots(country, countries4Plot, startdate, predict_range, \
                        startCase, opt, version, show=True, maxDate=maxDateStr, model=model)
else: 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    #calcula data máxima dos gráficos
    #200 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate='9/22/20'
    maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
    maxDateStr = maxDate.strftime("%-m/%-d/%y")   
    cp.covid_plots(country, countries4Plot,  startdate, predict_range, \
                       startCase, opt, version, show=True, maxDate=maxDateStr, model=model)